In [1]:
include("Intervalos.jl")
#include("Taylor9.jl")
using .Intervalos
#using .Taylor9

In [2]:
mutable struct Taylor{T<:Real}
    orden::Int64
    coef::Array{T, 1}
    function Taylor{T}(coef::Array{T,1}) where {T<:Real}
        #@show coef, typeof(coef)
        new(length(coef)-1, coef)
    end
end

In [3]:
I1 = Intervalo(2,5)
I2 = Intervalo(3,8)
I3 = Intervalo(0,3.5)
I4 = Intervalo(4.7,9.9)

I[4.7, 9.9]


In [4]:
import Base: +, -, *, /,^, getindex, setindex!,==

In [5]:
function getindex(f::Taylor, i::Int64) 
    f.coef[i + 1]
end

getindex (generic function with 192 methods)

In [6]:
function getindex(f::Taylor{T}, r::UnitRange{Int64}) where {T<:Real}
    newcoef = f.coef[(r[1] +1): (r[end] +1)]
    Taylor{T}(newcoef)
end

getindex (generic function with 193 methods)

In [7]:
function setindex!(f::Taylor, x::Real, i::Int64)
    f.coef[i + 1] = x
end

setindex! (generic function with 83 methods)

In [8]:
function ==(f::Taylor, g::Taylor)
    f.orden == g.orden && f.coef == g.coef  ? true : false
end

== (generic function with 143 methods)

In [9]:
function +(f::Taylor{T}, g::Taylor{T}) where {T<:Real}
    if f.orden == g.orden
        Taylor{T}(f.coef .+ g.coef)
    elseif f.orden < g.orden
        Taylor{T}(f.coef[1:f.orden + 1] .+ g.coef[1:f.orden + 1])
    else
        Taylor{T}(g.coef[1:g.orden + 1] .+ f.coef[1:g.orden + 1])
    end
end

+ (generic function with 167 methods)

In [10]:
function -(f::Taylor{T}, g::Taylor{T}) where {T<:Real}
    if f.orden == g.orden
        Taylor{T}(f.coef .- g.coef)
    elseif f.orden < g.orden
        Taylor{T}(f.coef[1:f.orden + 1] .- g.coef[1:f.orden + 1])
    else
        Taylor{T}(-g.coef[1:g.orden + 1] .+ f.coef[1:g.orden + 1])
    end
end

- (generic function with 180 methods)

In [11]:
typeof([2.0,3.0])

Array{Float64,1}

In [12]:
function *(f::Taylor{T}, g::Taylor{T}) where {T<:Real}
    f.orden != g.orden && error("Ordenes distintos")
    #∞newcoef = Array{T, 1}(f.orden + 1)
    newcoef = Array{T,1}(UndefInitializer(), f.orden + 1)
    for k in 0:f.orden 
        Taylori = 0.0
        for i in 0:k
            Taylori += f[i]*g[k - i]  
        end
        newcoef[k + 1] = Taylori 
    end
    Taylor{T}(newcoef)
end

* (generic function with 343 methods)

In [13]:
function /(f::Taylor{T}, g::Taylor{T}) where {T<:Real}
    f.orden != g.orden && error("Ordenes distintos")
    #newcoef = Array{T, 1}(f.orden + 1)
    newcoef = Array{T,1}(UndefInitializer(), f.orden + 1)
    for k in 0:f.orden 
        Taylori = f[k]
        for i in 0:k - 1
            Taylori -= (f[i]/g[i])*g[k - i]  
        end
        newcoef[k + 1] = Taylori/g[0] 
    end
    Taylor{T}(newcoef)
end

/ (generic function with 108 methods)

In [14]:
function ^(f::Taylor{T}, n::Int64) where {T<:Real}
    newTaylor = f
    for i in 1:n-1
        newTaylor *= f
    end
    newTaylor
end

^ (generic function with 64 methods)

In [15]:
@show I1 = Intervalo(2,5)
@show I2 = Intervalo(3,8)

I1 = Intervalo(2, 5) = I[2.0, 5.0]

I2 = Intervalo(3, 8) = I[3.0, 8.0]



I[3.0, 8.0]


In [16]:
#I = Taylor([1.0,2.0])
Ii = Taylor{Intervalo}([I1,I2])

@show Ii;

Ii = Taylor{Intervalo}(1, Intervalo[I[2.0, 5.0]
, I[3.0, 8.0]
])


In [17]:
@show Ii[0];

Ii[0] = I[2.0, 5.0]



In [18]:
Ii[1] = Intervalo(0,0)

@show Ii

@show Ii == Taylor{Intervalo}([Intervalo(2,5), Intervalo(0,0)])

Ii = Taylor{Intervalo}(1, Intervalo[I[2.0, 5.0]
, I[0.0, 0.0]
])
Ii == Taylor{Intervalo}([Intervalo(2, 5), Intervalo(0, 0)]) = true


true

In [19]:
@show I1 + I3, I1 - I3
@show I2 + I4, I2 - I4
@show Taylor{Intervalo}([I1,I2]) + Taylor{Intervalo}([I3,I4])
@show Taylor{Intervalo}([I1,I2]) - Taylor{Intervalo}([I3,I4]);

(I1 + I3, I1 - I3) = (I[2.0, 8.5]
, I[-1.5, 5.0]
)
(I2 + I4, I2 - I4) = (I[7.7, 17.9]
, I[-6.9, 3.3]
)
Taylor{Intervalo}([I1, I2]) + Taylor{Intervalo}([I3, I4]) = Taylor{Intervalo}(1, Intervalo[I[2.0, 8.5]
, I[7.7, 17.9]
])
Taylor{Intervalo}([I1, I2]) - Taylor{Intervalo}([I3, I4]) = Taylor{Intervalo}(1, Intervalo[I[-1.5, 5.0]
, I[-6.9, 3.3]
])


In [20]:
println(I1,I2,I3,I4)

I[2.0, 5.0]
I[3.0, 8.0]
I[0.0, 3.5]
I[4.7, 9.9]



In [21]:
@show Taylor{Intervalo}([I1,I2]) * Taylor{Intervalo}([I3,I4])
#@show Taylor{Intervalo}([I1,I2]) / Taylor{Intervalo}([I3,I4]);

Taylor{Intervalo}([I1, I2]) * Taylor{Intervalo}([I3, I4]) = Taylor{Intervalo}(1, Intervalo[I[0.0, 17.5]
, I[9.4, 77.5]
])


# El integrador numérico

In [100]:
function paso(x::Taylor{T}) where {T<:Real}
    p = length(x.coef) - 1
    (1.0e-32/(abs(x[p])))^(1/p)
end

function horner(P::Taylor{T}, x::Real) where {T<:Real}
    result = 0.0
    for i in P.orden:-1:0
        result = (result * x) + P{T}[i]
    end
    result
end

function serieTaylor(f,  x0::T, orden::Int64=16) where {T<:Real}
    newTaylor = Taylor{T}(Array{Intervalo,1}(UndefInitializer(),orden + 1))
    #newTaylor = Taylor{T}(Array{Intervalo, 1}(orden + 1))
    newTaylor[0] = x0
    newTaylor[1] = f(x0)
    for i in 2:orden
        x0 = newTaylor[0:i]
        newTaylor[i] = (f(x0)[i-1])/(i)
    end
    newTaylor
end


function serieTaylor(f,  x0::T, t0::Real, orden::Int64=16) where {T<:Real}
    newTaylor = Taylor{T}(Array{Intervalo,1}(UndefInitializer(),orden + 1))
    #newTaylor = Taylor{T}(Array{Intervalo, 1}(orden + 1))
    newTaylor[0] = x0
    newTaylor[1] = f(x0, t0)
    for i in 2:orden
        x0 = newTaylor[0:i]
        newTaylor[i] = (f(x0, t0)[i-1])/(i)
    end
    newTaylor
end

function taylorIntegration(f, x0::T, t0::Real ,orden::Int64=32) where {T<:Real}
    Ta = serieTaylor(f, x0, t0, orden)
    t = paso(Ta)
    x = horner(Ta, t)
    x, t + t0
end

function Picard(f, x0::T, t::Real, orden::Int64=32) where {T<:Real}
    x0new = Taylor{T}(Array{Intervalo, 1}(orden + 1))
    x0new[0] = x0
    x0new[1] = f(x0)
    for i in 2: orden 
        x0 = x0new[0:i]
        x0new[i] = (f(x0)[i-1])/(i)
    end
    horner(x0new, t)
end

Picard (generic function with 2 methods)

In [102]:
f(x) = x^2  
X0 = Intervalo(0.0,1/9)

a = serieTaylor(f, X0)

Taylor{Intervalo}(16, Intervalo[I[0.0, 0.111111]
, I[0.0, 0.0123457]
, I[0.0, 0.00137174]
, I[0.0, 0.000152416]
, I[0.0, 1.69351e-5]
, I[0.0, 1.88168e-6]
, I[0.0, 2.09075e-7]
, I[0.0, 2.32306e-8]
, I[0.0, 2.58117e-9]
, I[0.0, 2.86797e-10]
, I[0.0, 3.18664e-11]
, I[0.0, 3.54071e-12]
, I[0.0, 3.93412e-13]
, I[0.0, 4.37124e-14]
, I[0.0, 4.85694e-15]
, I[0.0, 5.3966e-16]
, I[0.0, 5.99622e-17]
])

## Prueba con `TaylorSeries`

In [72]:
using TaylorSeries

In [73]:
∫⬩dt(u::Taylor1) = integrate(u)

∫⬩dt (generic function with 1 method)

In [74]:
function taylor_step(f, u0)

    u = copy(u0)
    unew = u0 + ∫⬩dt(f(u))

    while unew != u
        u = unew
        unew = u0 + ∫⬩dt(f(u))   # Picard iteration
    end
    
    return u
end

taylor_step (generic function with 1 method)

In [97]:
f(x) = x^2

degree = 16

u0 = Taylor1([1/9], degree)

soln = taylor_step(f, u0)

 0.1111111111111111 + 0.012345679012345678 t + 0.0013717421124828531 t² + 0.00015241579027587256 t³ + 1.6935087808430286e-5 t⁴ + 1.8816764231589206e-6 t⁵ + 2.0907515812876894e-7 t⁶ + 2.3230573125418767e-8 t⁷ + 2.5811747917131966e-9 t⁸ + 2.867971990792441e-10 t⁹ + 3.186635545324934e-11 t¹⁰ + 3.5407061614721493e-12 t¹¹ + 3.934117957191276e-13 t¹² + 4.371242174656973e-14 t¹³ + 4.85693574961886e-15 t¹⁴ + 5.396595277354286e-16 t¹⁵ + 5.996216974838096e-17 t¹⁶ + 𝒪(t¹⁷)

In [108]:
a[16].sup

5.996216974838096e-17